# param recovery for both model

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import confusion_matrix
from joblib import Parallel, delayed
import matplotlib.tri as tri
import matplotlib.colors as mcolors
from scipy.interpolate import griddata
from scipy.interpolate import RBFInterpolator
import matplotlib.ticker as mticker
import itertools
from sklearn.metrics import r2_score
import optuna


# important directories

In [2]:
output_dir = "27_RL_agent_TDlearn_output_both_param_recovery"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added'
folder_path_colors_numbers = '13_RL_agent_TDlearn_output/model_behavior'


df_participants = []
df_colors_numbers = []


def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_excel(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True) 
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".xlsx")
        find_matching_csv(folder_path_colors_numbers, df_colors_numbers)


In [3]:
for df in df_participants:
    df['block_type'] = None

    df.loc[df['block'] == 1, 'block_type'] = 'uniform'     # Block 1 is uni
    df.loc[df['block'] == 4, 'block_type'] = 'mix'     # Block 4 is mix

    # For blocks 2 and 3, set based on distribution
    df.loc[(df['block'] == 2) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 2) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    



for i in range(len(df_participants)):
    myCard = df_participants[i]['myCard']
    yourCard = df_participants[i]['yourCard']
    distributions = df_participants[i]['distribution']
    block_type = df_participants[i]['block_type']
    
    for df_list in [ df_colors_numbers]:
        df_list[i]['myCard'] = myCard
        df_list[i]['yourCard'] = yourCard
        df_list[i]['distribution'] = distributions
        df_list[i]['block_type'] = block_type

In [4]:
for df in df_colors_numbers:
    df['model_choices'] = df['model_choices'].replace({1: 'arrowup', 0: 'arrowdown'})


In [5]:
for df in df_colors_numbers:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 'arrowup') or (my < your and choice == 'arrowdown')):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes

In [6]:
df_participants[2]

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk,block_type
0,2609,uniform,789,lose,4,2,335,9.5,0,response,arrowdown,1,0,1,0.375,uniform
1,597,uniform,853,win,9,4,407,10,1,response,arrowup,1,0,1,0.000,uniform
2,188,uniform,904,win,4,7,504,10.5,2,response,arrowdown,1,0,1,0.375,uniform
3,423,uniform,916,win,2,4,434,11,3,response,arrowdown,1,0,1,0.125,uniform
4,549,uniform,806,win,5,7,287,11.5,4,response,arrowdown,1,0,1,0.500,uniform
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,761,high,913,win,7,8,382,80,125,response,arrowdown,4,0,1,0.447,mix
266,596,low,921,win,4,3,318,80.5,83,response,arrowup,4,0,1,0.385,mix
267,414,low,950,win,2,7,335,81,77,response,arrowdown,4,0,1,0.243,mix
268,1371,uniform,842,win,6,4,615,81.5,35,response,arrowup,4,0,1,0.375,mix


In [7]:
df_colors_numbers[2]

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val,myCard,yourCard,distribution,block_type,outcome
0,arrowup,0,10.5,9.5,"[[[0.022019422455794066, 0.02059341150055373],...",4,2,uniform,uniform,win
1,arrowup,1,11.0,10.0,"[[[0.022019422455794066, 0.02059341150055373],...",9,4,uniform,uniform,win
2,arrowdown,0,11.5,10.5,"[[[0.022019422455794066, 0.02059341150055373],...",4,7,uniform,uniform,win
3,arrowdown,0,12.0,11.0,"[[[0.022019422455794066, 0.02059341150055373],...",2,4,uniform,uniform,win
4,arrowup,0,11.5,11.5,"[[[0.022019422455794066, 0.02059341150055373],...",5,7,uniform,uniform,lose
...,...,...,...,...,...,...,...,...,...,...
265,arrowdown,0,78.0,80.0,"[[[0.4991753037517269, 0.02059341150055373], [...",7,8,high,mix,win
266,arrowup,1,78.5,80.5,"[[[0.4991753037517269, 0.02059341150055373], [...",4,3,low,mix,win
267,arrowdown,0,79.0,81.0,"[[[0.4991753037517269, 0.02059341150055373], [...",2,7,low,mix,win
268,arrowup,1,79.5,81.5,"[[[0.4991753037517269, 0.02059341150055373], [...",6,4,uniform,mix,win


In [8]:
participants = [os.path.splitext(file)[0].replace("task_data_", "")
    for file in os.listdir(folder_path_participants) if file.endswith('.xlsx')]

In [9]:
actions = { "arrowdown": 0, "arrowup": 1}
distributions_map = { "uniform": 0, "low": 1,  "high": 2}
card_numbers = list(range(1, 10))

# policy_table = percentage_matrix 


Q_table_init = np.random.normal(0, 0.01, (len(card_numbers), len(distributions_map), len(actions)))


Q_table = Q_table_init.copy()

#############################################################################################
# having a q-table that starts with 0! this was not a good initilization so i changed it.
# Q_table = np.zeros((len(distributions_map), len(actions)))  # 3 distributions × 2 actions
#############################################################################################

# print("policy: \n",np.shape(policy_table))
print("\n Q_table: \n",np.shape(Q_table))




 Q_table: 
 (9, 3, 2)


In [10]:
def epsilon_greedy(Q_values, epsilon):    
    probs = np.full_like(Q_values, epsilon / Q_values.shape[-1], dtype=float)  # initialize with exploration probability
    best_actions = np.argmax(Q_values, axis=-1)  # find the best action for each state
    np.put_along_axis(probs, np.expand_dims(best_actions, axis=-1), 1 - epsilon + (epsilon / Q_values.shape[-1]), axis=-1)
    return probs



def train_rescorla_wagner(df, alpha, beta, Q_init=None):
    if Q_init is None:
        Q_init = Q_table.copy()
    Q_values = Q_init.copy()
    q_value_pairs = []
    choices = []
    predicted_probs = []
    distributions = []
    card_numbers = []
    
    for _, row in df.iterrows():
        action = actions[row["model_choices"]] 
        distribution = distributions_map[row["distribution"]] 
        card_number = row["myCard"]-1 # since I'm using this as an index! I need to do -1 to make the 1 to 9 cards come to 0 to 8
        reward = 0.5 if row["outcome"] == "win" else -0.5


        probs = epsilon_greedy(Q_values, beta)
        predicted_probs.append(probs[card_number][distribution][action])
        
        prediction_error = reward - Q_values[card_number][distribution][action]
        Q_values[card_number][distribution][action] += alpha * prediction_error
        
        q_value_pairs.append(Q_values.copy())
        choices.append(action)
        distributions.append(distribution)
        card_numbers.append(card_number)
        

    return np.array(q_value_pairs), np.array(choices), np.array(predicted_probs), np.array(distributions), np.array(card_numbers)


# this is for the sake of parallel computing
def compute_log_likelihood(alpha, beta, df_all, Q_table):
    Q_init_participant = Q_table.copy()
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, alpha, beta, Q_init=Q_init_participant.copy())
    
    predicted_probs = np.clip(predicted_probs, 1e-6, 1)  # prevent log(0)
    log_likelihood = np.sum(np.log(predicted_probs))
    
    return (alpha, beta, log_likelihood)


In [11]:
num_of_samples = 100
# num_of_samples = 1000
alpha_min = 0.01
alpha_max = 1
beta_min = 0.01
beta_max  = 1
alpha_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
beta_samples = np.random.uniform(beta_min, beta_max + np.finfo(float).eps, num_of_samples)

In [12]:
BIC_models = []
AIC_models = []
best_alpha_models = []
best_beta_models = []
accuracy_models = []
precision_models = []
sensitivity_recall_models = []
specificity_models = []
f1_score_models = []
mcFadden_r2_models = []
r2_models = []

for idx, df_all in enumerate(df_colors_numbers):
    print(f"Processing participant {idx + 1} of {len(df_colors_numbers)}")
    Q_init_participant = Q_table.copy()
    
    def objective(trial):
        alpha = trial.suggest_float("alpha", alpha_min, alpha_max)
        beta  = trial.suggest_float("beta", beta_min, beta_max)

        # negative log-likelihood (Optuna minimises)
        _, _, ll = compute_log_likelihood(alpha, beta,
                                        df_all,
                                        Q_init_participant.copy())
        return -ll

    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=400, n_jobs=-1)


    best_alpha = study.best_params["alpha"]
    best_beta  = study.best_params["beta"]
    best_log_likelihood = -study.best_value

    # keep this for plotting later
    results_df = study.trials_dataframe()
    results_df["alpha"] = results_df["params_alpha"]
    results_df["beta"]  = results_df["params_beta"]
    results_df["log_likelihood"] = -results_df["value"]

    # model prediction 
    
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, best_alpha, best_beta, Q_init=Q_init_participant.copy())
    
    
    predicted_choices = []
    for trial in range(len(card_numbers)):
        test_action_probs = epsilon_greedy(q_values[trial], best_beta)
        p_arrowup = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowup"]]
        p_arrow_down = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowdown"]]
        # choosing 1 or 0 based on the softmax probabilities:
        predicted_choice = int(p_arrowup >= 0.5)  # 1 if up-prob ≥ 0.5 else 0
        predicted_choices.append(predicted_choice)

    # finding out model total reward based on the model's predicted choices
    total_reward = [] 
    for i in range(len(predicted_choices)):
        if len(total_reward)> 0:
            last_reward = total_reward[-1]  #  the last reward value
        else:
            last_reward = 10 # initial reward is $10
        
        if ((df_all.loc[i, 'myCard'] > df_all.loc[i, 'yourCard'] and predicted_choices[i] == 1) or
            (df_all.loc[i, 'myCard'] < df_all.loc[i, 'yourCard'] and predicted_choices[i] == 0)):
            total_reward.append(last_reward + 0.5)
        else:
            total_reward.append(last_reward - 0.5)

    
   
       # confusion matrix:
    conf_matrix = confusion_matrix(choices, predicted_choices)
    TN, FP, FN, TP = conf_matrix.ravel()  # unpacking the confusion matrix
    # acc
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    # precision: From the ones that we’ve announced them as up/down, which ones are really up/down?
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    # recall or sensitivity : true positive rate
    sensitivity_recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    # specificity : true negative rate
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    # f1 Score
    f1_score = 2 * (precision * sensitivity_recall) / (precision + sensitivity_recall) if (precision + sensitivity_recall) != 0 else 0

    
    # bayes information criterion:
    n_trials = len(df_all)
    k = 2  # number of free parameters: alpha and beta
    BIC = k * np.log(n_trials) - 2 * best_log_likelihood # this is BIC formula based on the log lkelihode I found before

    

    # Akaike  information criterion(AIC):
    AIC = 2 * k - 2 * best_log_likelihood 


    # mcFadden r-squared:
    p_null = np.mean(choices)  # probability of choosing "1" in the dataset
    log_likelihood_null = np.sum(choices * np.log(p_null) + (1 - choices) * np.log(1 - p_null))
    mcFadden_r2 = 1 - (best_log_likelihood / log_likelihood_null)

    # r-squared
    r2 = r2_score(choices, predicted_choices)
    print(best_alpha)
    print(best_beta)
    
    # saving models evaluation variables:
    best_alpha_models.append(best_alpha)
    best_beta_models.append(best_beta)
    BIC_models.append(BIC)
    AIC_models.append(AIC)
    accuracy_models.append(accuracy)
    precision_models.append(precision)
    sensitivity_recall_models.append(sensitivity_recall)
    specificity_models.append(specificity)
    f1_score_models.append(f1_score)
    mcFadden_r2_models.append(mcFadden_r2)
    r2_models.append(r2)





[I 2025-06-13 13:17:14,466] A new study created in memory with name: no-name-8c8702f8-de1e-4ac8-90c6-fae177be04ad


Processing participant 1 of 38


[I 2025-06-13 13:17:15,424] Trial 6 finished with value: 199.51243994842554 and parameters: {'alpha': 0.652563608183823, 'beta': 0.3484023507183486}. Best is trial 6 with value: 199.51243994842554.
[I 2025-06-13 13:17:16,020] Trial 7 finished with value: 185.30192510381704 and parameters: {'alpha': 0.29068444609538513, 'beta': 0.9759289316457623}. Best is trial 7 with value: 185.30192510381704.
[I 2025-06-13 13:17:16,126] Trial 24 finished with value: 179.93512263160346 and parameters: {'alpha': 0.6627995505357521, 'beta': 0.531790815581247}. Best is trial 24 with value: 179.93512263160346.
[I 2025-06-13 13:17:16,144] Trial 11 finished with value: 175.53346214814462 and parameters: {'alpha': 0.7872565904312403, 'beta': 0.7572510534161042}. Best is trial 11 with value: 175.53346214814462.
[I 2025-06-13 13:17:16,173] Trial 12 finished with value: 220.17930485499622 and parameters: {'alpha': 0.07096403893640192, 'beta': 0.2667528194418312}. Best is trial 11 with value: 175.53346214814462.

0.46087539496442936
0.6978949669383202
Processing participant 2 of 38


[I 2025-06-13 13:17:41,853] Trial 1 finished with value: 154.7085110697899 and parameters: {'alpha': 0.9223938965041685, 'beta': 0.5520816747553112}. Best is trial 2 with value: 149.54644352416852.
[I 2025-06-13 13:17:42,425] Trial 6 finished with value: 161.47004436972557 and parameters: {'alpha': 0.48080328697693314, 'beta': 0.7476347086799356}. Best is trial 2 with value: 149.54644352416852.
[I 2025-06-13 13:17:42,522] Trial 3 finished with value: 160.0682475105441 and parameters: {'alpha': 0.7883469252096557, 'beta': 0.35537144318794356}. Best is trial 2 with value: 149.54644352416852.
[I 2025-06-13 13:17:42,698] Trial 10 finished with value: 151.97690182181253 and parameters: {'alpha': 0.059573790840446776, 'beta': 0.6465957594229531}. Best is trial 2 with value: 149.54644352416852.
[I 2025-06-13 13:17:42,724] Trial 7 finished with value: 168.15240729826039 and parameters: {'alpha': 0.9730689505736767, 'beta': 0.8225342443758599}. Best is trial 2 with value: 149.54644352416852.
[I

...

0.06979563944707357
0.5565163180732328
Processing participant 37 of 38


[I 2025-06-13 13:33:09,814] Trial 24 finished with value: 293.9354015072497 and parameters: {'alpha': 0.6208051018473227, 'beta': 0.09957611494841158}. Best is trial 24 with value: 293.9354015072497.
[I 2025-06-13 13:33:09,892] Trial 4 finished with value: 266.3766864277812 and parameters: {'alpha': 0.42749625998467977, 'beta': 0.1338630817262876}. Best is trial 4 with value: 266.3766864277812.
[I 2025-06-13 13:33:09,910] Trial 26 finished with value: 228.84067095274068 and parameters: {'alpha': 0.0529289890058046, 'beta': 0.2174412375876671}. Best is trial 26 with value: 228.84067095274068.
[I 2025-06-13 13:33:09,920] Trial 15 finished with value: 187.50131391443492 and parameters: {'alpha': 0.08994477760031636, 'beta': 0.4262112200953687}. Best is trial 15 with value: 187.50131391443492.
[I 2025-06-13 13:33:10,224] Trial 0 finished with value: 226.86405471690946 and parameters: {'alpha': 0.22420097848158316, 'beta': 0.22347656016943132}. Best is trial 15 with value: 187.5013139144349

0.2601729725331008
0.6690557475861848
Processing participant 38 of 38


[I 2025-06-13 13:33:34,418] Trial 9 finished with value: 193.153417347325 and parameters: {'alpha': 0.06422734589540424, 'beta': 0.564350677443492}. Best is trial 9 with value: 193.153417347325.
[I 2025-06-13 13:33:34,420] Trial 1 finished with value: 203.11641980550849 and parameters: {'alpha': 0.6491010866176032, 'beta': 0.5332850309494539}. Best is trial 9 with value: 193.153417347325.
[I 2025-06-13 13:33:34,422] Trial 0 finished with value: 187.82757751123495 and parameters: {'alpha': 0.2639117956223755, 'beta': 0.7063254444514238}. Best is trial 0 with value: 187.82757751123495.
[I 2025-06-13 13:33:34,425] Trial 5 finished with value: 434.4984204119048 and parameters: {'alpha': 0.4220313907429943, 'beta': 0.05374332044740758}. Best is trial 0 with value: 187.82757751123495.
[I 2025-06-13 13:33:34,430] Trial 2 finished with value: 271.02239871006896 and parameters: {'alpha': 0.6260167092206207, 'beta': 0.2365748852315812}. Best is trial 0 with value: 187.82757751123495.
[I 2025-06-

0.020677832590421243
0.7857998398759786


# now saving the model evaluation values

In [13]:
df_models_evaluation = pd.DataFrame({
    "participants": participants,
    "best_alpha": best_alpha_models,
    "best_beta": best_beta_models,
    "BIC": BIC_models,
    "AIC": AIC_models,
    "accuracy": accuracy_models,
    "precision": precision_models,
    "sensitivity_recall": sensitivity_recall_models,
    "specificity": specificity_models,
    "f1_score": f1_score_models,
    "mcFadden_r2": mcFadden_r2_models,
    "r2": r2_models
})

file_path = os.path.join(output_dir, "models_evaluation_greedy.csv")
df_models_evaluation.to_csv(file_path, index=False)